O objetivo desse projeto é realizar a classificação de operações em cartões de créditos como **fraude** ou **legítimo**.

*Nas versões 1 e 2 eu utilizei Redes Neurais para realizar a predição.* 

*Apliquei um ***threshold*** *(limiar) de* ***0.9***, *ou seja, até esse valor, considera-se* ***negativo*** *para fraude e o restante* ***positivo*** *para fraude.*

*O* ***Recall*** *ficou em* ***0.5437*** *e o* ***falso positivo*** *foi cerca de* ***11%*** *de todas as operação que não eram fraude.*

Descobri um desempenho superior no **XGBClassifier**, portanto, escolhi mudar o algorítimo.

Com um **threshold** de **0.863** e **Recall** de **0.5480** (tentei manter os Recalls próximos para avaliar os falsos positivos), o **falso positivo** ficou em torno de **4%**.

Iniciaremos com as importações necessárias.

In [ ]:
# importação das bibliotecas e ferramentas de ML e de métricas
# manipulação de dados
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import FunctionTransformer, StandardScaler
from category_encoders import TargetEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# machine learning
from xgboost import XGBClassifier

# métricas
from sklearn.metrics import roc_auc_score # Competition metric
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# joblib
from joblib import dump, load

Agora, faremos o carregamento dos dados csv no python e depois visualizaremos o **head()**

In [ ]:
df = pd.read_csv("/kaggle/input/playground-series-s3e4/train.csv")
df.head()

Após isso, utilizaremos o **describe()** para termos uma visualização da sumarização das colunas.

Como pederemos ver a seguir, os dados são todos **numéricos contínuos**.

In [ ]:
df.describe()

Aqui, a função **nunique()** servirá para nos dizer quantos dados únicos temos em cada coluna. Dessa forma poderemos saber se há algum possível dado **categórico** convertido para **numérico**.

Mas como veremos a seguir, são todos **numéricos** mesmo.

In [ ]:
df.nunique()

Feito isso, iremos verificar se há algum **null** com as funções **isnull().sum()**.

In [ ]:
df.isnull().sum()

Nessa etapa o **info()** irá nos mostrar algumas informações sobre as colunas, como por exemplo, o tipo de dado e a quantidade de dados válidos em cada coluna.

In [ ]:
df.info()

Agora, analisaremos a quantidade de dados que são fraudes (saída 1) e a quantidade que não são fraudes (saída 0) por tipo de saída (*target*).

Isso é importante, pois como veremos a seguir, há uma quantidade muito grande de *target* 0, ou seja, uma quantidade muito grande de dados que não são fraude e isso acaba por enviezar o modelo de **machine learning**.

In [ ]:
contagem_classes = df['Class'].value_counts()
print(contagem_classes)

Vamos dividir em dois **DataFrames** os dados de fraude e os dados legítimos.

In [ ]:
df_fraud = df[df['Class'] == 1]
df_legit = df[df['Class'] == 0]

print(df_fraud['Class'].value_counts())
print(df_legit['Class'].value_counts())

Dando, continuidade, agora coletamos uma amostra dos dados não-fraude na mesma quantidade dos dados fraude para igualar a quantidade de cada um, a fim de não enviezar o modelo.

Após isso, concatenaremos as duas tabelas em apenas uma.

In [ ]:
df_legit = df_legit.sample(n=469, random_state=1)
df_reshaped = pd.concat([df_legit,df_fraud])
df_reshaped['Class'].value_counts()

A partir de agora, iniciaremos nosso pipeline.

A baixo, começaremos criando uma lista com todas as *features* numéricas, que nesse caso serão todas as *features* que temos no nosso DataFrame.

In [ ]:
# Início do pipeline de treinamento
# Criação de lista para numerical e Categorical features

numerical_features = []

for c in df_reshaped.columns:
  if c != 'id' and c != 'Class':
    if df_reshaped[c].dtype != "dtype('object')":
      numerical_features.append(c)

numerical_features

Faremos o mesmo para a lista com todas as *features* e com o *target*.

In [ ]:
# Selecionando features e target

features = list(df_reshaped.columns)
for c in ['id', 'Class']:
  features.remove(c)

target = [
    'Class'
]

features

Agora separaremos os dados em **treino** e **teste** para realização do treinamento do modelo.

In [ ]:
X = df_reshaped.drop(columns= ['id','Class'])
y = df_reshaped['Class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Veremos se os dados estão dividos adequadamente.

In [ ]:
contClassTrain = y_train.value_counts()
print(contClassTrain)
contClassTest = y_test.value_counts()
print(contClassTest)

Com a separação realizada, podemos começar a escrever o *pipeline* de preprocessamento.

Como temos apenas dados numéricos, faremos o preprocessamento apenas para ele.

Utilizaremos o método de padronização escalar e substituição dos dados nulos pela mediana do conjunto.

Aplicaremos o modelo de classificação ***XGBoost*** e depois o fit (treino) do modelo.

In [ ]:
# Preprocessamento de colunas numéricas
numeric_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Combinando pré-processadores de colunas numéricas e categóricas
preprocessor = ColumnTransformer([
    ('numeric', numeric_transformer, numerical_features)
])


# Criando o pipeline com etapas de pré-processamento e modelo
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('lrg', XGBClassifier(learning_rate=0.02, n_estimators=1000, objective='binary:logistic',
                    nthread=-1, colsample_bytree=0.6,
                    gamma=1, max_depth=5,min_child_weight=10,subsample=0.8))
])

# Resetando o índice
X_train.reset_index(drop=True, inplace=True)
y_train.reset_index(drop=True, inplace=True)

In [ ]:
# Treinando o pipeline
pipeline.fit(X_train, y_train)

Separação da df principal para teste.

In [ ]:
# separação de dados treino e teste
X = df[features]
y = df[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Após o treinamento, faremos a previsão dos dados de teste.

In [ ]:
y_pred1 = pipeline.predict_proba(X_test)[:,1]
roc_auc_score(y_test, y_pred1)

Agora, verificaremos os resultados do teste ajustando o threshold para um valor adequado.

In [ ]:
# visualização dos resultados obtidos
threshold = 0.863

y_pred_binary = list((y_pred1 > threshold).astype(int))

def printer(y_true, y_pred):
    print('Accuracy:', accuracy_score(y_true, y_pred))
    print('Precision:', precision_score(y_true, y_pred))
    print('Recall:', recall_score(y_true, y_pred))
    print('F1:', f1_score(y_true, y_pred))
    print('Confusion Matrix:\n', confusion_matrix(y_true, y_pred))

printer(y_test, y_pred_binary)